In [36]:
#import csv from the dataset
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import random as rnd

from sklearn.model_selection import cross_val_score

# machine learning
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier
sns.set(font_scale=1)
plt.style.use('ggplot')


train=pd.read_csv('../trainwDSM.csv')
validate=pd.read_csv('../validatewDSM.csv')
test=pd.read_csv('../testwDSM.csv')

train.shape, validate.shape, test.shape

((1929971, 34), (643324, 34), (643324, 34))

In [37]:
#train = train.query('SUB1 <= 10').query('SUB2 <= 10')
#validate = validate.query('SUB1 <= 10').query('SUB2 <= 10')

drop_list = ['SUB2', 'YEAR', 'STFIPS', 'Unnamed: 0']
X_train = train.drop(drop_list, axis=1)
Y_train = train["SUB2"]
X_validate = validate.drop(drop_list, axis=1)
Y_validate = validate["SUB2"]
X_test  = test.drop(drop_list, axis=1)
X_train.shape, X_validate.shape, X_test.shape



((1929971, 30), (643324, 30), (643324, 30))

In [4]:
X_train.head(10)

,AGE,GENDER,RACE,ETHNIC,EDUC,EMPLOY,LIVARAG,ARRESTS,SUB1,NUMSUBS,...,STIMFLG,BENZFLG,TRNQFLG,BARBFLG,SEDHPFLG,INHFLG,OTCFLG,OTHERFLG,ALCDRUG,DSMCRIT
112,5,1,5,6,4,2,1,0,18,2,...,0,0,0,0,0,0,1,1,2,2
122,6,2,5,5,3,3,3,0,10,2,...,0,1,0,0,0,0,0,0,2,8
141,5,2,5,5,4,3,3,0,10,3,...,0,1,0,0,0,0,0,0,3,8
408,6,2,5,5,3,3,3,0,10,2,...,0,0,0,0,0,0,0,1,2,8
727,9,2,5,5,5,3,3,0,10,3,...,0,0,0,0,0,0,0,0,3,8
840,7,2,5,5,3,3,1,0,13,2,...,0,1,0,0,0,0,0,0,2,8
1120,7,1,5,5,3,2,3,0,13,2,...,0,1,0,0,0,0,0,1,2,11
1294,7,1,5,1,2,3,1,0,20,2,...,0,0,0,0,0,0,0,1,2,20
1407,9,2,5,5,2,3,3,0,10,3,...,0,1,0,0,0,0,0,0,2,8
1571,9,1,21,5,3,3,1,0,10,3,...,0,0,0,0,0,0,0,1,2,8


In [38]:
# Decision Tree

decision_tree = DecisionTreeClassifier()
decision_tree.fit(X_train, Y_train)
Y_pred = decision_tree.predict(X_validate)
#print cross_val_score(decision_tree, X_validate, Y_validate)

In [39]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import hinge_loss
# Decision Tree score
print round(accuracy_score(Y_pred, Y_validate) * 100, 2)
print round(f1_score(Y_pred, Y_validate, average='macro') * 100, 2)
print round(f1_score(Y_pred, Y_validate, average='micro') * 100, 2)
#print round(hinge_loss(Y_pred, Y_validate) * 100, 2)


90.48
77.44
90.48


In [40]:
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(Y_validate, Y_pred, labels=[1,2,3,4,5,7,10])

In [42]:
import itertools

def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

# Compute confusion matrix
cnf_matrix = cm #confusion_matrix(y_test, Y_pred)
class_names = ['NONE','ALCOHOL', 'COCAINE', 'MARIJUANA', 'HEROIN', 'OPIATES','METH']

np.set_printoptions(precision=2)

# Plot non-normalized confusion matrix
plt.figure()
plot_confusion_matrix(cnf_matrix, class_names,
                      title='Confusion matrix, without normalization')
plt.savefig('cnf matrix', dpi=150)

# Plot normalized confusion matrix
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=class_names, normalize=True,
                      title='Normalized confusion matrix')

#plt.figure(figsize=(16,8))
plt.savefig('cnf matrix norm', dpi=150)

Confusion matrix, without normalization
[[306053   1413    944    493     52    188     29]
 [   639  65606   2265   4465    379    976    744]
 [   507   2425  43156   3427    541    858    465]
 [   249   4718   4439  82927    483   1397    999]
 [    76    417    638    497   9600    206     91]
 [   128   1187    863   1522    250  24932    277]
 [    30    924    555   1485    136    335  11784]]
Normalized confusion matrix
[[  9.90e-01   4.57e-03   3.05e-03   1.59e-03   1.68e-04   6.08e-04
    9.38e-05]
 [  8.51e-03   8.74e-01   3.02e-02   5.95e-02   5.05e-03   1.30e-02
    9.91e-03]
 [  9.87e-03   4.72e-02   8.40e-01   6.67e-02   1.05e-02   1.67e-02
    9.05e-03]
 [  2.62e-03   4.96e-02   4.66e-02   8.71e-01   5.07e-03   1.47e-02
    1.05e-02]
 [  6.59e-03   3.62e-02   5.54e-02   4.31e-02   8.33e-01   1.79e-02
    7.90e-03]
 [  4.39e-03   4.07e-02   2.96e-02   5.22e-02   8.57e-03   8.55e-01
    9.50e-03]
 [  1.97e-03   6.06e-02   3.64e-02   9.74e-02   8.92e-03   2.20e-02
    7.7

# Logistic Regression

In [ ]:
logreg = LogisticRegression()
logreg.fit(X_train, Y_train)
#Y_pred = logreg.predict(X_test)
acc_log = round(logreg.score(X_validate, Y_validate) * 100, 2)
print acc_log
#print cross_val_score(logreg, X_validate, Y_validate)

In [6]:
# Perceptron

perceptron = Perceptron()
perceptron.fit(X_train, Y_train)
#Y_pred = perceptron.predict(X_test)
acc_perceptron = round(perceptron.score(X_validate, Y_validate) * 100, 2)
print acc_perceptron
#print cross_val_score(perceptron, X_validate, Y_validate)

48.18
[ 0.0019632   0.48187181  0.4817682 ]


90.45
[ 0.9002173   0.9003101   0.89980087]


In [10]:
# Stochastic Gradient Descent

sgd = SGDClassifier()
sgd.fit(X_train, Y_train)
#Y_pred = sgd.predict(X_test)
acc_sgd = round(sgd.score(X_validate, Y_validate) * 100, 2)
print acc_sgd
#print cross_val_score(sgd, X_validate, Y_validate)

/Users/arc/Codes/DM-Lab/dmlab/lib/python2.7/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


11.94


In [11]:
# Gaussian Naive Bayes

gaussian = GaussianNB()
gaussian.fit(X_train, Y_train)
#Y_pred = gaussian.predict(X_test)
acc_gaussian = round(gaussian.score(X_validate, Y_validate) * 100, 2)
print acc_gaussian
#print cross_val_score(gaussian, X_validate, Y_validate)

48.21


In [ ]:
knn = KNeighborsClassifier(n_neighbors = 2)
knn.fit(X_train, Y_train)
#Y_pred = knn.predict(X_test)
acc_knn = round(knn.score(X_validate, Y_validate) * 100, 2)
print acc_knn
#print cross_val_score(knn, X_validate, Y_validate)

In [12]:
# Random Forest (slow)

random_forest = RandomForestClassifier(n_estimators=100)
random_forest.fit(X_train, Y_train)
#Y_pred = random_forest.predict(X_test)
random_forest.score(X_train, Y_train)
acc_random_forest = round(random_forest.score(X_validate, Y_validate) * 100, 2)
print acc_random_forest
#print cross_val_score(random_forest, X_validate, Y_validate)

91.91
